In [181]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [182]:
train_data = pd.read_csv('Train_Data.csv')
test_data = pd.read_csv('Test_Data.csv')

In [183]:
X_train = train_data.drop(['attack'], axis=1)
y_train = train_data['attack']
X_test = test_data.copy()

In [184]:
categorical_cols = ['protocoltype', 'service', 'flag', 'lastflag']
numerical_cols = [col for col in X_train.columns if col not in categorical_cols]

In [185]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [186]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [187]:
model = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=32))
])

In [188]:
label_mapping = {'normal': 0, 'neptune': 1}
y_train = y_train.map(label_mapping)

In [189]:
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=32)

In [190]:
model.fit(X_train_split, y_train_split)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['duration', 'srcbytes',
                                                   'dstbytes', 'land',
                                                   'wrongfragment', 'urgent',
                                                   'hot', 'numfailedlogins',
                                                   'loggedin', 'numcompromised',
                                                   'rootshell', 'suattempted',
                                                   'numroot',
                                                   'numfilecreations',
                                                   'numshells',
                                                   'numaccessfiles',
                                                   'numoutboundcmds',
                                                   'ishostlogin',
                                                   'isguestlogin', 'count',
                                                   'srvcount', 'serrorrate',
                                                   'srvserrorrate',
                                                   'rerrorrate',
                                                   'srvrerrorrate',
                                                   'samesrvrate', 'diffsrvrate',
                                                   'srvdiffhostrate',
                                                   'dsthostcount',
                                                   'dsthostsrvcount', ...]),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['protocoltype', 'service',
                                                   'flag', 'lastflag'])])),
                ('classifier', RandomForestClassifier(random_state=32))])

In [191]:
y_pred_val = model.predict(X_val)
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))
print("Classification Report:\n", classification_report(y_val, y_pred_val))
print("Validation Accuracy:", accuracy_score(y_val, y_pred_val))

Confusion Matrix:
 [[10797     0]
 [    0  6572]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     10797
           1       1.00      1.00      1.00      6572

    accuracy                           1.00     17369
   macro avg       1.00      1.00      1.00     17369
weighted avg       1.00      1.00      1.00     17369

Validation Accuracy: 1.0


In [192]:
predictions = model.predict(X_test)

In [193]:
submission = pd.DataFrame({'attack': predictions})
submission.to_csv('submission.csv', index=False)